In [20]:
#Importar librerías
from vivienda_cdmx.vivicdmx import lamudi,easybroker
import pandas as pd
import time
import random
from matplotlib import pyplot as plt
import folium
from folium.plugins import HeatMap,MarkerCluster
import branca



#### Scrapeo de datos de los portales inmobiliarios

In [3]:
# Listado
portales = [ easybroker, lamudi]
# df vacía
df = pd.DataFrame()

# proceso de scrapeo
for portal in portales:
    try:
        # Tiempo de inicio
        print("Proceso iniciado a las ", time.strftime("%H:%M:%S"),"del día", time.strftime("%d/%m/%y"))
        
        # Ejecuta la función de scraping
        df_portal = portal()  
        df = pd.concat([df, df_portal])
        
        # Records
        print("Número de registros: ", len(df))
        
    except Exception as e:
        # Si ocurre un error, lo imprime
        print(f"Ocurrió un error: {e}")
    
    finally:
        
        print("Proceso terminado a las ", time.strftime("%H:%M:%S"),"del día", time.strftime("%d/%m/%y"))
        print("--------------------------------------------------")
        
        # Sleep for a random time before the next iteration
        time.sleep(random.randint(1, 3))

    
df

Proceso iniciado a las  13:25:48 del día 07/04/24


Scrapeando Easybroker: 100%|██████████| 100/100 [07:26<00:00,  4.46s/it]


Número de registros:  2375
Proceso terminado a las  13:33:14 del día 07/04/24
--------------------------------------------------
Proceso iniciado a las  13:33:16 del día 07/04/24


Scrapeando Lamudi: 100%|██████████| 100/100 [05:40<00:00,  3.40s/it]


Número de registros:  5381
Proceso terminado a las  13:40:16 del día 07/04/24
--------------------------------------------------


,oferta,precio,mts_const,direccion,bathrooms,recamaras,lat,lon,fuente,fecha_consulta
0,venta casa lindavista 5620000,5620000.0,180.0,"Casa en Valle del Tepeyac, Gustavo A. Madero",2.0,3.0,19.4882202,-99.1421561,easybroker,2024-04-07 13:33:14.984453
1,venta casa lindavista 6340000,6340000.0,310.0,"Casa en Valle del Tepeyac, Gustavo A. Madero",3.0,4.0,19.4882202,-99.1421561,easybroker,2024-04-07 13:33:14.984453
2,venta casa olivar de los padres av toluca. al...,6990000.0,213.0,"Casa en Olivar de los Padres, Álvaro Obregón",3.0,4.0,19.3376687,-99.2374879,easybroker,2024-04-07 13:33:14.984453
3,venta residencia lomas de chapultepec 54000000...,54000000.0,962.0,"Casa en Lomas de Chapultepec I Sección, Miguel...",6.0,4.0,19.4187281,-99.2151321,easybroker,2024-04-07 13:33:14.984453
4,venta casa rancho san francisco en fracc. priv...,33000000.0,538.0,Casa en Rancho San Francisco Pueblo San Bartol...,5.0,3.0,19.3360421,-99.2595795,easybroker,2024-04-07 13:33:14.984453
...,...,...,...,...,...,...,...,...,...,...
3003,casa en venta en lomas de chapultepec i seccion,69000000.0,606.0,NaN,2,3,19.409126,-99.22934,lamudi,2024-04-07 13:40:16.892400
3004,casa en condominio en venta en sta fe,5100000.0,200.0,NaN,2,2,19.3497738,-99.3033707,lamudi,2024-04-07 13:40:16.892400
3005,casa en venta en bosque de las lomas,42890000.0,900.0,NaN,4,4,19.399868,-99.25373,lamudi,2024-04-07 13:40:16.892400
3006,casa en venta en san jose insurgentes,9000000.0,99.0,NaN,1,2,19.366766,-99.188283,lamudi,2024-04-07 13:40:16.892400


In [4]:
df["fuente"].unique()

array(['easybroker', 'lamudi'], dtype=object)

#### Limpieza de datos

In [5]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros sin latitud y longitud
    df=df[df['lat'].notna()]
    df=df[df['lon'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('renta')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio'],keep='first')

    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts_const']

    return df

In [6]:
df_limpia=limpia_datos(df)
df_limpia

,oferta,precio,mts_const,direccion,bathrooms,recamaras,lat,lon,fuente,fecha_consulta,precio_m2
0,venta casa lindavista 5620000,5620000.0,180.0,"Casa en Valle del Tepeyac, Gustavo A. Madero",2.0,3.0,19.4882202,-99.1421561,easybroker,2024-04-07 13:33:14.984453,31222.222222
1,venta casa lindavista 6340000,6340000.0,310.0,"Casa en Valle del Tepeyac, Gustavo A. Madero",3.0,4.0,19.4882202,-99.1421561,easybroker,2024-04-07 13:33:14.984453,20451.612903
2,venta casa olivar de los padres av toluca. al...,6990000.0,213.0,"Casa en Olivar de los Padres, Álvaro Obregón",3.0,4.0,19.3376687,-99.2374879,easybroker,2024-04-07 13:33:14.984453,32816.901408
3,venta residencia lomas de chapultepec 54000000...,54000000.0,962.0,"Casa en Lomas de Chapultepec I Sección, Miguel...",6.0,4.0,19.4187281,-99.2151321,easybroker,2024-04-07 13:33:14.984453,56133.056133
4,venta casa rancho san francisco en fracc. priv...,33000000.0,538.0,Casa en Rancho San Francisco Pueblo San Bartol...,5.0,3.0,19.3360421,-99.2595795,easybroker,2024-04-07 13:33:14.984453,61338.289963
...,...,...,...,...,...,...,...,...,...,...,...
5372,casa en venta en cuajimalpa de morelos,5050000.0,189.0,NaN,2,1,19.3692,-99.29089,lamudi,2024-04-07 13:40:16.892400,26719.576720
5375,casa en venta en bosques de las lomas,19500000.0,320.0,NaN,3,3,19.392918,-99.255433,lamudi,2024-04-07 13:40:16.892400,60937.500000
5376,casa en venta en lomas de chapultepec i seccion,69000000.0,606.0,NaN,2,3,19.409126,-99.22934,lamudi,2024-04-07 13:40:16.892400,113861.386139
5379,casa en venta en san jose insurgentes,9000000.0,99.0,NaN,1,2,19.366766,-99.188283,lamudi,2024-04-07 13:40:16.892400,90909.090909


In [7]:
df_limpia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4277 entries, 0 to 5380
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   oferta          4277 non-null   object        
 1   precio          4277 non-null   float64       
 2   mts_const       4161 non-null   float64       
 3   direccion       1916 non-null   object        
 4   bathrooms       4075 non-null   object        
 5   recamaras       4155 non-null   object        
 6   lat             4277 non-null   object        
 7   lon             4277 non-null   object        
 8   fuente          4277 non-null   object        
 9   fecha_consulta  4277 non-null   datetime64[ns]
 10  precio_m2       4161 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 401.0+ KB


In [21]:
#Mapa de folium de la ciudad de México con df_limpia
mapa = folium.Map(location=[19.4326, -99.1332], zoom_start=12, tiles="http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}", attr="Google")
agg_data = df_limpia.groupby(['lat', 'lon']).size().reset_index(name='counts')

# Finding vmin and vmax for the heatmap
vmin_value = agg_data['counts'].min()
vmax_value = agg_data['counts'].max()

HeatMap(data=agg_data[['lat', 'lon', 'counts']], radius=8, max_zoom=14,name="Mapa de calor",overlay=True, control=True,show=True,

        gradient={0.0: 'yellow', 0.5: 'orange', 1.0: 'red'}).add_to(mapa)

cmap = branca.colormap.LinearColormap(
    colors=['yellow', 'orange', 'red'],
    index=[vmin_value, (vmin_value + vmax_value) / 2, vmax_value],
    vmin=vmin_value, vmax=vmax_value,
    caption="Viviendas"
)

mapa.add_child(cmap)


# Create a FeatureGroup for the markers
marker_cluster = MarkerCluster(name="Viviendas").add_to(mapa)
for i in range(0,len(df_limpia)):
    folium.Marker([df_limpia.iloc[i]['lat'], df_limpia.iloc[i]['lon']], popup=df_limpia.iloc[i]['oferta'], name="Viviendas").add_to(marker_cluster)
    
folium.LayerControl().add_to(mapa)

#Salvar el mapa
mapa.save("mapa_cdmx.html")

mapa